In [129]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import importlib
import seaborn as sns

name_columns =[  
    "ID пассажира",
    "Выжил",
    "Класс билета", 
    "Имя",
    "Пол",
    "Возраст",
    "Количество братьев/сестер/супругов на борту",
    "Количество родителей/детей на борту",
    "Номер билета",
    "Стоимость",
    "Номер каюты",
    "Порт посадки"
    ]

data = pd.read_csv('./data_tms.csv', header=0, delimiter=r",", names=name_columns)

data.head(5)




,ID пассажира,Выжил,Класс билета,Имя,Пол,Возраст,Количество братьев/сестер/супругов на борту,Количество родителей/детей на борту,Номер билета,Стоимость,Номер каюты,Порт посадки
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [130]:
np.shape(data)

(891, 12)

In [132]:
mean_age = data.groupby("Выжил")["Возраст"].mean()
print(mean_age)

Выжил
0    30.626179
1    28.343690
Name: Возраст, dtype: float64


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


df = data

df = df.dropna(subset=['Пол', 'Порт посадки', 'Возраст', 'Стоимость'])

df = pd.get_dummies(df, columns=['Пол', 'Порт посадки'], drop_first=True)

y = df['Выжил']
X = df.drop(columns=['Выжил', 'Имя', 'Номер билета', 'Номер каюты', 'ID пассажира'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



model_under = LinearRegression()
model_under.fit(X_train, y_train)

y_pred_under = (model_under.predict(X_test) >= 0.5).astype(int)
acc_under = accuracy_score(y_test, y_pred_under)
print(f'Недообучение — Accuracy: {acc_under:.4f}')



model_over = DecisionTreeClassifier(max_depth=None)
model_over.fit(X_train, y_train)

y_pred_over = model_over.predict(X_test)
acc_over = accuracy_score(y_test, y_pred_over)
print(f'Переобучение — Accuracy: {acc_over:.4f}')


model_balanced = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
model_balanced.fit(X_train, y_train)

y_pred_balanced = model_balanced.predict(X_test)
acc_balanced = accuracy_score(y_test, y_pred_balanced)
print(f'Корректная модель — Accuracy: {acc_balanced:.4f}')


data = pd.get_dummies(data,columns = ['Пол'], drop_first=False)

data_int = data.drop(columns=['Имя','Возраст','Выжил','Номер билета','Номер каюты', "Порт посадки"])

first_model = LinearRegression()
first_model.fit(data_int,data['Выжил'])

first_model.coef_

predict_first = first_model.predict(data_int)
y_pred_binary = (predict_first >= 0.5).astype(int)


accuracy = accuracy_score(y_true=data['Выжил'], y_pred=y_pred_binary)
print(accuracy)



Недообучение — Accuracy: 0.7762
Переобучение — Accuracy: 0.7063
Корректная модель — Accuracy: 0.7832
0.7957351290684624


In [143]:
import math
import numpy as np






def sigmoid(z):
  return 1/(1 + np.exp(-z))

def predict(x, weight):
  z = weight[0]
  for i in range(len(x)):
    z += weight[i+1] * x[i]
  return sigmoid(z)

def loss (X, y, weights):
  m = len(y)
  total = 0
  for i in range(m):
    p = predict(X[i], weights)
    total += -(y[i] * math.log(p) + (1 - y[i]) * math.log(1-p))
  return total/m

def train(X, y, lr=0.1, epochs=1000):
    m, n = len(X), len(X[0])
    weights = [0.0] * (n + 1)
    
    for _ in range(epochs):
        grad = [0.0] * (n + 1)
        for i in range(m):
            p = predict(X[i], weights)
            error = p - y[i]
            grad[0] += error  # bias
            for j in range(n):
                grad[j + 1] += error * X[i][j]
        
        for j in range(n + 1):
            weights[j] -= lr * grad[j] / m
    
    return weights


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

name_columns =[  
    "ID пассажира", "Выжил", "Класс билета", "Имя", "Пол", "Возраст",
    "Количество братьев/сестер/супругов на борту", "Количество родителей/детей на борту",
    "Номер билета", "Стоимость", "Номер каюты", "Порт посадки"
]

data = pd.read_csv('./data_tms.csv', header=0, delimiter=",", names=name_columns)

data = data.dropna(subset=["Пол", "Порт посадки", "Возраст", "Стоимость"])

data = pd.get_dummies(data, columns=["Пол", "Порт посадки"], drop_first=True)

# Нормализация числовых признаков
scaler = StandardScaler()
X_num = scaler.fit_transform(data[[
    "Класс билета", "Возраст",
    "Количество братьев/сестер/супругов на борту",
    "Количество родителей/детей на борту",
    "Стоимость"
]])

# Категориальные признаки
X_cat = data.drop(columns=[
    "ID пассажира", "Имя", "Номер билета", "Номер каюты",
    "Класс билета", "Возраст", "Количество братьев/сестер/супругов на борту",
    "Количество родителей/детей на борту", "Стоимость", "Выжил"
]).values

X = np.hstack([X_num, X_cat])
y = data["Выжил"].tolist()




weights = train(X, y, lr=0.1, epochs=10000)

print("Веса:", weights)
for x in X:
    print(f"x={x}, p={predict(x, weights):.3f}")

y_pred = [int(predict(x, weights) >= 0.5) for x in X]
acc = sum(int(y_pred[i] == y[i]) for i in range(len(y))) / len(y)
print(f"Accuracy: {acc:.4f}")


Веса: [np.float64(1.4931931605945434), np.float64(-1.0038277503557067), np.float64(-0.6279161397576537), np.float64(-0.3377049061240095), np.float64(-0.051179295561326596), np.float64(0.07636908055296676), np.float64(-2.636104071246666), np.float64(-0.7900193271608581), np.float64(-0.3929032733843661)]
x=[ 0.90859974 -0.52766856  0.52251079 -0.50678737 -0.51637992  1.
  0.          1.        ], p=0.091
x=[-1.48298257  0.57709388  0.52251079 -0.50678737  0.69404605  0.
  0.          0.        ], p=0.926
x=[ 0.90859974 -0.25147795 -0.55271372 -0.50678737 -0.50362035  0.
  0.          1.        ], p=0.627
x=[-1.48298257  0.36995092  0.52251079 -0.50678737  0.35032585  0.
  0.          1.        ], p=0.903
x=[ 0.90859974  0.36995092 -0.55271372 -0.50678737 -0.50125747  1.
  0.          1.        ], p=0.075
x=[-1.48298257  1.68185631 -0.55271372 -0.50678737  0.3269333   1.
  0.          1.        ], p=0.296
x=[ 0.90859974 -1.9086216   2.67295981  0.66474707 -0.25504498  1.
  0.          1. 

In [ ]:
from sklearn.linear_model import LogisticRegression

X_sklearn = data.drop(columns=['Имя','Возраст','Выжил','Номер билета','Номер каюты', "Порт посадки"])
y_sklearn = data['Выжил']

model = LogisticRegression(max_iter=1000)
model.fit(X_sklearn, y_sklearn)

print(f"Accuracy (sklearn): {model.score(X_sklearn, y_sklearn):.4f}")

Accuracy (sklearn): 0.7969
